In [1]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error


plt.style.use('bmh')

In [2]:
path_s = 'data/stock/BBAS3.SA.csv'


ticket = path_s.split('/')[1].split('.')[0]
dfs = pd.read_csv(path_s,index_col='Date')
dfs.dropna(inplace=True)
print(dfs.shape)
dfs.head()

(5739, 6)


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2000-01-04,2.733333,2.740000,2.603333,2.603333,1.064754,314100.0
2000-01-05,2.600000,2.633333,2.506666,2.630000,1.075661,478800.0
2000-01-06,2.630000,2.666666,2.600000,2.666666,1.090656,205200.0
2000-01-07,2.666666,2.666666,2.606666,2.606666,1.066117,394200.0
2000-01-10,2.730000,2.753333,2.666666,2.750000,1.124740,549000.0


In [3]:
df=dfs[['Close']]

In [4]:
future_days = 940
df['prediction']=df[['Close']].shift(-future_days)
df.head()

C:\Users\015025631\AppData\Local\Temp\ipykernel_25944\790076301.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['prediction']=df[['Close']].shift(-future_days)


,Close,prediction
Date,,
2000-01-04,2.603333,6.516666
2000-01-05,2.630000,6.576666
2000-01-06,2.666666,6.516666
2000-01-07,2.606666,6.613333
2000-01-10,2.750000,6.566666


In [5]:
X = np.array(df.drop(['prediction'],1))[:-future_days]
print(X)

[[ 2.603333]
 [ 2.63    ]
 [ 2.666666]
 ...
 [52.09    ]
 [52.939999]
 [51.48    ]]


C:\Users\015025631\AppData\Local\Temp\ipykernel_25944\332875867.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = np.array(df.drop(['prediction'],1))[:-future_days]


In [6]:
y = np.array(df['prediction'])[:-future_days]

In [7]:
#split 
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2)


In [8]:
#regressores
tree= DecisionTreeRegressor().fit(x_train,y_train)
lr = LinearRegression().fit(x_train,y_train)
rf = RandomForestRegressor(n_estimators=400,max_depth=190).fit(x_train,y_train)

In [9]:
#últimas linhas de x do futuro
x_future = df.drop(['prediction'],1)[:-future_days]
x_future = x_future.tail(future_days)
x_future = np.array(x_future)


C:\Users\015025631\AppData\Local\Temp\ipykernel_25944\2824249725.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  x_future = df.drop(['prediction'],1)[:-future_days]


In [10]:
tree_prediction = tree.predict(x_future)
lr_prediction = lr.predict(x_future)
rf_prediction = rf.predict(x_future)


In [11]:

predictions = lr_prediction

valid = df[X.shape[0]:]
valid['Predictions'] = predictions
df_fig = df.copy()
df_fig = df_fig.join(valid.add_prefix('lr_'))
#df_fig.index = df.index

fig1 = px.line(df_fig,x=df_fig.index,y=['Close','lr_Close','lr_Predictions'],color_discrete_sequence=['blue','red','black'])
fig1.show()
print(mean_squared_error(valid['Close'], predictions))

C:\Users\015025631\AppData\Local\Temp\ipykernel_25944\851579842.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid['Predictions'] = predictions


126.00657513771502


In [12]:
predictions = tree_prediction

valid = df[X.shape[0]:]
valid['Predictions'] = predictions
df_fig = df.copy()
df_fig = df_fig.join(valid.add_prefix('tree_'))
#df_fig.index = df.index

fig2 = px.line(df_fig,x=df_fig.index,y=['Close','tree_Close','tree_Predictions'],color_discrete_sequence=['blue','red','purple'])
fig2.show()
print(mean_squared_error(valid['Close'], predictions))

C:\Users\015025631\AppData\Local\Temp\ipykernel_25944\1309530533.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



36.061875351064145


In [13]:
predictions = rf_prediction

valid = df[X.shape[0]:]
valid['Predictions'] = predictions
df_fig = df.copy()
df_fig = df_fig.join(valid.add_prefix('rf_'))

fig3 = px.line(df_fig,x=df_fig.index,y=['Close','rf_Close','rf_Predictions'],color_discrete_sequence=['blue','red','yellow'])
fig3.show()
print('erro quadrático médio:',mean_squared_error(valid['Close'], predictions))
print('valores:','\n',valid['Close'],'\n', predictions)

C:\Users\015025631\AppData\Local\Temp\ipykernel_25944\3750258370.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



erro quadrático médio: 37.80622892883156
valores: 
 Date
2019-06-06    52.790001
2019-06-07    52.360001
2019-06-10    51.869999
2019-06-11    52.900002
2019-06-12    52.259998
                ...    
2023-03-13    38.150002
2023-03-14    37.509998
2023-03-15    37.549999
2023-03-16    37.950001
2023-03-17    37.299999
Name: Close, Length: 940, dtype: float64 
 [46.61920673 40.39682143 43.7815431  49.85873719 43.65506437 42.91465393
 49.85873719 43.49298616 35.34581844 39.30373343 44.52698503 39.05233024
 45.59065989 40.56054382 48.59517925 49.85873719 43.6090825  50.13684339
 51.83929558 45.27196775 44.32028956 38.47532861 42.03878248 37.99504796
 37.79855792 46.72014982 42.04966077 41.69149905 38.34387422 43.76418357
 29.61071858 43.33132289 45.17969634 45.14230296 42.51176666 51.83929558
 38.12751488 39.79785154 42.79760095 31.8269632  39.54712728 43.10987916
 34.37586142 51.83929558 36.27085128 30.80941901 45.8674286  39.96100577
 36.23575462 42.91465393 39.33128426 27.59852429 34.

In [14]:
fig1.add_trace(fig2.data[2]).add_trace(fig3.data[2])